This notebook attempts to use The RCA framework to do QC on image segmentation, but using the exact same CNN used for doing the segmentation as the starting point.

In [ ]:
from tensorflow.keras.models import model_from_json,clone_model
from tensorflow.keras.optimizers import Adam

from mask_utils import iou

from network_utils import augmentImageSequence, gpu_memory_limit

import numpy as np

import os

from sklearn.model_selection import train_test_split

import copy

import matplotlib.pyplot as plt

In [ ]:
gpu_memory_limit(8000)

Load data and model, and get them sorted in the same way as in the notebooks used to train models (i.e. same train/test split, random seed etc)

In [ ]:
DataDir = './data/pericardial/wsx_round2/'

#load data - these files created by extract_dcm_for_wsx.ipynb
X = np.load(os.path.join(DataDir,'X.npy'))
Y = np.load(os.path.join(DataDir,'Y.npy')).astype('float')
pxArea = np.load(os.path.join(DataDir,'pxSize.npy'))
pxSpacing = np.sqrt(pxArea)

#ensure the shape is correct arrays saved were rank 3, so this changes to rank 4 (last dimension represents channels)
X = X.reshape([*X.shape,1])
Y = Y.reshape([*Y.shape,1])

#do train/test split!
X_train, X_test, Y_train, Y_test,pxArea_train,pxArea_test,pxSpacing_train,pxSpacing_test = train_test_split(X, Y, pxArea,pxSpacing, test_size=0.2,random_state=101)

# X = X[:200,:,:,:]
# Y = Y[:200,:,:,:]

#
# M = X.shape[0]
# MTest = X_test.shape[0]

Now, need to load a model which can be used for the RCA

In [ ]:
#pick a model, just need one to play with.
modelBaseName = './data/models/mrunet_2020-04-07_09:59' #THIS MODEL IS NOT THE BEST ONE BUT HAS BEEN SELECTED TO GIVE A WIDE SPREAD IN IOU ON TRAIN AND TEST SETS

#load the model archistecture
with open( modelBaseName + '.json', 'r') as json_file:
    model = model_from_json( json_file.read() )
    
#get the weights
model.load_weights(modelBaseName + '.h5')    

NOTE! RCA uses a network output to train a classifier which can be applied to other labelled data. I plan to use as a starting point *the actual network* that was used to generate the segmentation. However, this will not work in the obvious way - training a network on its own output will result in the cost function ==0, and thus all gradients ==0. 

However, thresholding the network output first will work! as the network will never output 0/1, but some numbers close to them.

In [ ]:
OPT = Adam(learning_rate = 1e-2,
           beta_1 = 0.9,
           beta_2 = 0.999,
           amsgrad = False
          )


def retrain_model(model,x,y):#,optimizer):
    
    ''''''
    
    assert np.all(model.input_shape[1:] == x.shape[1:]) and np.all(model.input_shape[1:] == y.shape[1:]),'image input shape and model input do not match - have you reshaped the image correctly?'
    
    assert x.shape[0] == 1 and y.shape[0]==1, 'you can only do RCA on one image at a time!'
    
    #threshold mask so that it can be used as a target for CNN
    y = y > 0.5
    
    #make a complete local copy of the model so that it is not modified globally
    weights = model.get_weights()
    model_local = clone_model(model)
    model_local.set_weights(weights)
    
    model_local.compile(optimizer = OPT, 
                        loss = 'binary_crossentropy'
                       )
    
    fit_history = model_local.fit(x=x,
                                  y=y,
                                  epochs = 100, #THINK ABOUT ME
                                  steps_per_epoch= 1, #obvs
                                  verbose=0
                                 )
    
    return model_local,fit_history

In [ ]:
def evaluate_model(model,X_val,Y_val):
    
    '''this function takes a model (presumably retrained on a predicted mask in order to do RCA) and evaluates it on the set of masks which are known'''

#     assert np.all(X.shape==Y.shape),'looks like you have mismatched your images and masks'
#     assert X.shape[0]>1,'you should only use this on more than one image. Are you doing what you think youre doing?
    
    
    Y_pred = model.predict(X_val)
    
    ious = np.array([iou(Y_pred[m],Y_val[m]) for m in range(Y_pred.shape[0])])
    
    return ious

In [ ]:
def predict_and_RCA_evaluate(model,x,X_val,Y_val):

    assert np.all(model.input_shape[1:] == x.shape[1:]),'image input shape and model input do not match - have you reshaped the image correctly?'
    assert x.shape[0] == 1, 'you can only do RCA on one image at a time!'
    
    y = model.predict(x)
    
    rca_model,model_history = retrain_model(model,x,y)
    
    ious = evaluate_model(rca_model,X_val,Y_val)
    
    return ious,model_history

First, we should look at the iou spread over the whole thing... The IOUs of all of the data we plan to use, to evaluate the *original* model

In [ ]:
trueIOUs = evaluate_model(model,X_test,Y_test)

plt.hist(trueIOUs,bins = np.arange(0,1.05,0.05))

Now, lets just get a single datapoint to play with... 

In [ ]:
#select an example image
np.random.seed(7)
egInd = np.random.randint(X_test.shape[0])

#get the IOU that we want to predict...
trueIOU = trueIOUs[egInd]

#get the actual image out and shaped correctly
egX = X_test[egInd,:,:].reshape(1,*model.input_shape[1:])

#get all images EXCEPT that one, from both X and Y
mask = np.ones(X_test.shape[0],dtype=bool)
mask[egInd] = False
X_val = X_test[mask,:,:,:]
Y_val = Y_test[mask,:,:,:]

predictedIOUs,fitHistory = predict_and_RCA_evaluate(model,egX,X_val,Y_val)

So, the below plot represents a single example - and we can maybe have some intuitions about how to summarise the distribution etc. However, in order to sort the hyperparameters for the refit, we need to examine how the IOUs of the evaluation set change during the refitting process? 
What properties should these $\Delta$IOUs exhibit for a well-refitted model?
 - they should change, presumably downwards?
 - should they generally move towards the true IOU of the example?
 - should they remain in the same order?
 - What should the spread of $\Delta$ look like? It should not be uniform
 - The relative changes of the IOUs change depend on how the test point relates to the distribution of the 
      - if it is close, then the IOUs should change relatively little
      - it it is far, they should go down a lot
      - it is very unlikely that they should ever increase!
      
So, now we need to look at a picture of pre/post fit IOUs for the evaluation set...

In [ ]:
plt.figure(figsize = (15,5))

plt.subplot(1,3,1)
plt.plot(fitHistory.history['loss'])


plt.subplot(1,3,2)
plt.hist(predictedIOUs,bins= np.arange(0,1.05,0.05),density=True,label = 'IOU of evaluation examples after retraining')

plt.plot([trueIOU,trueIOU],plt.ylim(),c='r',label = 'true IOU')
plt.xlabel('iou')
plt.ylabel('probability density')
plt.legend()


plt.subplot(1,3,3)
plt.scatter(trueIOUs[mask],predictedIOUs,label = 'evaluation set')
plt.xlim([0,1])
plt.ylim([0,1])

plt.plot([0,1],[trueIOU,trueIOU],c='r',label = 'true IOU')
plt.plot([0,1],[0,1],c='k',label = 'line of unity')
plt.xlabel('original IOU')
plt.ylabel('IOU after retraining')

plt.legend()


So, now I will try and do this in more systematic manner, allowing hyperparameter tuning for the optimizer, number of epochs, and optional argument for data augmentation

In [ ]:
def retrain_model(model, x, y, optimizer, loss = 'binary_crossentropy', epochs = 10, dataGenArgs=None):
    
    '''retrain a model using a single datapoint, with inputs for various hyperparameters including the '''
    
    assert np.all(model.input_shape[1:] == x.shape[1:]) and np.all(model.input_shape[1:] == y.shape[1:]),'image input shape and model input do not match - have you reshaped the image correctly?'
    
    assert x.shape[0] == 1 and y.shape[0]==1, 'you can only do RCA on one image at a time!'
    
    #threshold mask so that it can be used as a target for CNN
    y = y > 0.5
    
    #make a complete local copy of the model so that it is not modified globally
    weights = model.get_weights()
    model_local = clone_model(model)
    model_local.set_weights(weights)
    
    model_local.compile(optimizer = optimizer, loss = loss)
    
    if dataGenArgs == None: #default option - just use the original image.
        model_local.fit(x=x,
                        y=y,
                        epochs = epochs, #THINK ABOUT ME
                        steps_per_epoch= 1, #obvs
                        verbose=0
                       )
    else: #if a dict is provided with arguments for data augmentation
        model_local.fit(augmentImageSequence(x,y,dataGenArgs,batchSize=1),
                        epochs = epochs, #THINK ABOUT ME
                        steps_per_epoch= 1, #obvs
                        verbose=0
                       )
    
    return model_local


def predict_and_RCA_evaluate(model,x,X_val,Y_val,optimizer,loss='binary_crossentropy',epochs=10,dataGenArgs = None):

    assert np.all(model.input_shape[1:] == x.shape[1:]),'image input shape and model input do not match - have you reshaped the image correctly?'
    assert x.shape[0] == 1, 'you can only do RCA on one image at a time!'
    
    #get the predictions from the model
    y = model.predict(x)
    
    #use the predictions to retrain the model
    rca_model = retrain_model(model, x, y, optimizer, loss = loss, epochs = epochs, dataGenArgs=dataGenArgs)
    
    #check the locally-retrained model against the evaluation dataset
    ious = evaluate_model(rca_model,X_val,Y_val)
    
    return ious

Additionally, there are considerations about which evaluation set should be used. It seems to me that thw whole point of this is to do with looking for similarities to the training set - so this should be used for evaluation

In [ ]:
#some hyperparameters for testing... taken from mrunet_dev.ipynb

dataGenArgs = dict(rotation_range=10,
                   width_shift_range=0.1,
                   height_shift_range=0.1,
                   shear_range=0.1,
                   zoom_range=0.1,
                   horizontal_flip=False, #DO NOT FLIP THE IMAGES FFS
                   vertical_flip=False,
                   fill_mode='nearest',
                   data_format= 'channels_last',
                   featurewise_center=False,
                   featurewise_std_normalization=False,
                   zca_whitening=False,
                  )


OPT = Adam(learning_rate = 3e-3,
           beta_1 = 0.9,
           beta_2 = 0.999,
           amsgrad = False
          )

EPOCHS = 100

In [ ]:
#get the true IOU for all of the test set
trueIOUs = evaluate_model(model,X_test,Y_test)

MTest =  X_test.shape[0]
M = X_train.shape[0]

predIOUs = np.zeros((MTest,M))

#loop over each test set example
for ind in range(MTest):

    predIOUs[ind,:] = predict_and_RCA_evaluate(model,egX,X_train,Y_train,optimizer=OPT,epochs=EPOCHS,dataGenArgs=dataGenArgs)
    

In [ ]:
from scipy.stats import pearsonr

In [ ]:
plt.figure(figsize = (5,15))

y = np.max(predIOUs,axis=1)
plt.subplot(3,1,1)
plt.plot([0,1],[0,1],c='k')
plt.scatter(trueIOUs,y)
plt.title(f'{pearsonr(trueIOUs,y)[0]:.02}')
plt.xlim([0,1])
plt.ylim([0,1])
plt.ylabel('predicted IOU (max)')                      

y = np.median(predIOUs,axis=1)            
plt.subplot(3,1,2)
plt.plot([0,1],[0,1],c='k')
plt.scatter(trueIOUs,y)
plt.title(f'{pearsonr(trueIOUs,y)[0]:.02}')
plt.xlim([0,1])
plt.ylim([0,1])
plt.ylabel('predicted IOU (median)')                      
  
y = np.mean(predIOUs,axis=1)
plt.subplot(3,1,3)
plt.plot([0,1],[0,1],c='k')
plt.scatter(trueIOUs,y)
plt.title(f'{pearsonr(trueIOUs,y)[0]:.02}')
plt.xlim([0,1])
plt.ylim([0,1])
plt.ylabel('predicted IOU (mean)')                      

            
plt.xlabel('true IOU')


So, that suuuuuucks. Give up.